In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pathlib
from glob import glob

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import *
from tensorflow.python.saved_model import signature_constants

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.exceptions import MlflowException
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec

from prefect import flow, task
from prefect.task_runners import SequentialTaskRunner

print(f'Pandas:{pd.__version__}, Numpy:{np.__version__}, Tensorflow:{tf.__version__}')

Pandas:1.3.4, Numpy:1.21.4, Tensorflow:2.7.0


In [18]:
def mlflow_setup():
    MLFLOW_TRACKING_URI ="sqlite:////home/pmspraju/tracking-server/mlflow.db" 
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

    return client

In [19]:
@task
def create_mlflow_experiment(experiment_name):
    try:
        experiment_id = mlflow.create_experiment(
            experiment_name,
            #artifact_location=Path.cwd().joinpath("mlruns").as_uri(),
            artifact_location='//home/pmspraju/tracking-server/mlruns/',
            tags={"version": "v1", "priority": "P1"},
        )
    except Exception as MlflowException:
        print(f"Experiment exists")
        experiment= mlflow.set_experiment(experiment_name)
        # Examine the experiment details.
        print("Experiment_id: {}".format(experiment.experiment_id))
        print("Name: {}".format(experiment.name))
        print("Artifact Location: {}".format(experiment.artifact_location))
        print("Tags: {}".format(experiment.tags))
        print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
        print("Last Updated timestamp: {}".format(experiment.last_update_time))

In [20]:
# Resize an image based on a scale factor. Take in to consideration that it receives an image represented as Numpy array.
def resize_image(image_array, factor):
    original_image = Image.fromarray(image_array)
    new_size = np.array(original_image.size) * factor
    new_size = new_size.astype(np.int32)
    new_size = tuple(new_size)

    resized = original_image.resize(new_size)
    resized = img_to_array(resized)
    resized = resized.astype(np.uint8)

    return resized

In [21]:
# Tightly crop an image. We need the image to fit nicely when we apply a sliding window to extract patches later. 
# SCALE is the actor we want the network to learn how to enlarge images by
def tight_crop_image(image, scale):
    height, width = image.shape[:2]
    width -= int(width % scale)
    height -= int(height % scale)

    return image[:height, :width]

In [22]:
# Reduce resolution of an imae by downsizing and then upsizing 
def downsize_upsize_image(image, scale):
    scaled = resize_image(image, 1.0 / scale)
    scaled = resize_image(scaled, scale / 1.0)

    return scaled

In [23]:
# Crop patches from input images. input_dim is the height&width of the images that is input to the network
def crop_input(image, x, y, input_dim):
    x_slice = slice(x, x + input_dim)
    y_slice = slice(y, y + input_dim)

    return image[y_slice, x_slice]

In [24]:
# Crop patches of output images. label_size is the height&width of the images output by the network. 
# pad is the number of pixels used as padding to ensure we are cropping the roi accurately
def crop_output(image, x, y, label_size, pad):
    y_slice = slice(y + pad, y + pad + label_size)
    x_slice = slice(x + pad, x + pad + label_size)
    #print(x, y , x_slice, y_slice)

    return image[y_slice, x_slice]


In [30]:
# method to prepare data
#@task
def prepare_data(path):
    SEED = 999
    SUBSET_SIZE = 100 #1500
    SCALE = 2.0
    INPUT_DIM = 33
    LABEL_SIZE = 21
    PAD = int((INPUT_DIM - LABEL_SIZE) / SCALE)
    STRIDE = 14
    
    np.random.seed(SEED)
    pattern = (path / 'images' / '*.png')
    file_patterns = str(pattern)
    dataset_paths = [*glob(file_patterns)]
    
    dataset_paths = np.random.choice(dataset_paths, SUBSET_SIZE)

    data = []; labels = []; cnt = 0

    for image_path in dataset_paths:
        image = load_img(image_path)
        image = img_to_array(image)
        image = image.astype(np.uint8); #Image.fromarray(image).show()
        image = tight_crop_image(image, SCALE); #Image.fromarray(image).show()
        scaled = downsize_upsize_image(image, SCALE); #Image.fromarray(scaled).show()

        height, width = image.shape[:2]
        #print(f'height:{height},width:{width}')
        
        for y in range(0, height - INPUT_DIM + 1, STRIDE):
            for x in range(0, width - INPUT_DIM + 1, STRIDE):
                crop = crop_input(scaled, x, y, INPUT_DIM); #Image.fromarray(crop).show()
                target = crop_output(image, x, y, LABEL_SIZE, PAD);#Image.fromarray(target).show()
                
                #cnt = cnt + 1
                
                data.append(crop)#data.append(np.array(crop).flatten()) #use np.reshape(fi,(33,33,3)) to read        
                labels.append(target)#labels.append(np.array(target).flatten());#use np.reshape(fi,(33,33,3)) to read  

                #fname = f'train/images/image_{y}_{x}.png' 
                #Image.fromarray(crop).save(os.path.join(path,fname))   

                #fname = f'train/labels/label_{y}_{x}.png'              
                #Image.fromarray(target).save(os.path.join(path,fname)) 
                #break
            #break
        #break
    #print(cnt)    
    return [data, labels]
    #pd.DataFrame({'image': data, 'label': labels})


In [11]:
# # Test data
# path = pathlib.Path('/mnt/c/Users/pmspr/Documents/Machine Learning/Courses/Tensorflow Cert/Data/dogscats')
# [data, labels] = prepare_data(path)
# print(f'shape:{np.array(labels).shape}')
# print(f'shape:{np.array(data).shape}')
# labels = tf.convert_to_tensor(labels, np.int32)
# print(labels.shape)

In [26]:
@task #- task cannot be called inside another task
def build_model(height, width, depth):
    input = Input(shape=(height, width, depth))

    x = Conv2D(filters=64, 
               kernel_size=(9,9),
               kernel_initializer='he_normal'
               ) (input)
    
    x = ReLU()(x)

    x = Conv2D(filters=32, 
               kernel_size=(1,1),
               kernel_initializer='he_normal'
                ) (x)

    x = ReLU()(x)

    output = Conv2D(filters=depth, 
                    kernel_size=(5,5),
                    kernel_initializer='he_normal'
                    ) (x)

    return Model(input, output)


In [27]:
@task
def train_model(path, dim,  data, labels, model):
    data = tf.convert_to_tensor(data, np.int32)
    labels = tf.convert_to_tensor(labels, np.int32)
    print(f'Data shape:{data.shape}, Labels shape"{labels.shape}')

    EPOCHS = 1 #12
    optimizer = Adam(learning_rate=1e-3, decay=1e-3 / EPOCHS)
    
    model.compile(loss='mse', optimizer=optimizer)
    BATCH_SIZE = 64
    
    model.fit(data, labels, batch_size=BATCH_SIZE, epochs = EPOCHS)

    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    input_schema = Schema([
                            TensorSpec(np.dtype(np.uint8), (-1, dim, dim, 3)),
                            ])
    output_schema = Schema([TensorSpec(np.dtype(np.uint8), (-1, 21, 21, 3))])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)


    mlflow.tensorflow.log_model(model=model,signature=signature,
                                artifact_path="tf-models")

    # write model summary
    summary = []
    model.summary(print_fn=summary.append)
    summary = "\n".join(summary)
    summary_path = os.path.join(path, "model_summary.txt")
    with open(summary_path, "w") as f:
        f.write(summary)
    mlflow.log_artifact(summary_path)
    os.remove(summary_path)

    # write model as json file
    model_json_path = os.path.join(path, "model.json")
    with open(model_json_path, "w") as f:
        f.write(model.to_json())
    mlflow.log_artifact(model_json_path)
    os.remove(model_json_path)

In [28]:
@flow(task_runner=SequentialTaskRunner())
def main():
    path = pathlib.Path('/mnt/c/Users/pmspr/Documents/Machine Learning/Courses/Tensorflow Cert/Data/dogscats')
    experiment_name = 'IMAGE-RESOLUTION'
    dim = 33
    client = mlflow_setup()
    create_mlflow_experiment(experiment_name)
    [data, labels] = prepare_data(path)
    model = build_model(dim, dim, 3)

    with mlflow.start_run() as run:
        print("MLflow:")
        print("  run_id:",run.info.run_id)
        print("  experiment_id:",run.info.experiment_id)
        mlflow.set_tag("version.mlflow", mlflow.__version__)
        mlflow.set_tag("version.tensorflow", tf.__version__)
        train_model(path, dim, data, labels, model)

In [31]:
if __name__ == "__main__":
    main()

13:01:50.852 | INFO    | prefect.engine - Created flow run 'psychedelic-elk' for flow 'main'
13:01:50.854 | INFO    | Flow run 'psychedelic-elk' - Using task runner 'SequentialTaskRunner'
13:01:50.866 | WARNING | Flow run 'psychedelic-elk' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
13:01:50.944 | INFO    | Flow run 'psychedelic-elk' - Created task run 'create_mlflow_experiment-863ae521-1' for task 'create_mlflow_experiment'
13:01:51.026 | INFO    | Task run 'create_mlflow_experiment-863ae521-1' - Finished in state Completed()


Experiment exists
Experiment_id: 3
Name: IMAGE-RESOLUTION
Artifact Location: //home/pmspraju/tracking-server/mlruns/
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Last Updated timestamp: 1674755884516


13:01:53.148 | INFO    | Flow run 'psychedelic-elk' - Created task run 'build_model-074892a9-0' for task 'build_model'


2023-01-29 13:01:54.394787: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-29 13:01:54.395033: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-72LU0U0G): /proc/driver/nvidia/version does not exist
2023-01-29 13:01:54.396675: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
13:01:54.495 | WARNING | tensorflow - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2023-01-29 13:01:54.749563: W tensorflow/python/util/util.cc:368] Sets are not currently considered s

INFO:tensorflow:Assets written to: ram://4712e70b-71b3-4340-87e1-8a8ff50432e2/assets


13:01:55.111 | INFO    | tensorflow - Assets written to: ram://4712e70b-71b3-4340-87e1-8a8ff50432e2/assets
13:01:55.176 | INFO    | Task run 'build_model-074892a9-0' - Finished in state Completed()


MLflow:
  run_id: 62d1d5f0b17f481bb14050047d8de998
  experiment_id: 3


13:02:00.344 | INFO    | Flow run 'psychedelic-elk' - Created task run 'train_model-7c866860-0' for task 'train_model'
2023-01-29 13:02:05.241738: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 881789436 exceeds 10% of free system memory.


Data shape:(67477, 33, 33, 3), Labels shape"(67477, 21, 21, 3)
1055/1055 [==============================] - 62s 59ms/step - loss: 253.0873
INFO:tensorflow:Assets written to: /tmp/tmphtxhn1s7/model/data/model/assets


13:04:37.107 | INFO    | tensorflow - Assets written to: /tmp/tmphtxhn1s7/model/data/model/assets
/home/pmspraju/.local/share/virtualenvs/imageResolution-c2hlZ-SW/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
13:04:41.072 | INFO    | Task run 'train_model-7c866860-0' - Finished in state Completed()
13:04:41.129 | INFO    | Flow run 'psychedelic-elk' - Finished in state Completed('All states completed.')


In [75]:
!jupyter nbconvert --to script imageResolution.ipynb

[NbConvertApp] Converting notebook imageResolution.ipynb to script
[NbConvertApp] Writing 6499 bytes to imageResolution.py


In [ ]:
# path = pathlib.Path('/mnt/c/Users/pmspr/Documents/Machine Learning/Courses/Tensorflow Cert/Data/dogscats')
# df = prepare_data(path)
# with open(os.path.join(path,'train.txt'), 'wb') as fp:
#     pickle.dump(df[0], fp, pickle.HIGHEST_PROTOCOL)
#np.savetxt(os.path.join(path,'train.txt'), df.values, fmt='%d')
# with open(os.path.join(path,'train.txt'), 'a') as f:
#     dfAsString = df.to_string(header=False, index=False)
#     f.write(dfAsString)